# Сравнение предложений

## Инструкции

1.Скачайте файл с предложениями

2.Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().

3.Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.

4.Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

5.Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.

6.Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.

7.Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

8.Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.

## Реализация

Импортируем необходимые библиотеки.

In [1]:
import re
import numpy as np
from scipy.spatial import distance

Считываем предложения из файла в список sentences.

In [2]:
with open('SimilarityOfTexts_sentences.txt','r') as file_obj:
    sentences = file_obj.readlines()

Разбиваем предложения на слова. Удаляем пустые слова.

In [3]:
for i in range(len(sentences)):
    sentences[i] = sentences[i].lower().strip('\n')
    sentences[i] = re.split('[^a-z]', sentences[i])
    for j in range(len(sentences[i])-1, -1, -1):
        if sentences[i][j] == '':
            del sentences[i][j]

Составляем список всех слов, встречающихся в предложениях. Сопоставляем каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях.

In [4]:
words = dict()
index = 0
for sentence in sentences:
    for word in sentence:
        if words.get(word, -1) == -1:
            words[word] = index
            index += 1

Создаём матрицу размера n * d, где n — число предложений. Примечание: значения n и d даны в условиях задания. 

In [5]:
m = 22
n = 254
matrix = np.zeros((22, 254))

Заполняем матрицу значениями. Элемент с индексом (i, j) в этой матрице равен количеству вхождений j-го слова в i-е предложение.

In [6]:
for i, sentence in enumerate(sentences):
    for word in sentence:
        j = words[word]
        matrix[i][j] += 1

Находим косинусное расстояние от предложения в самой первой строке до всех остальных с помощью функции scipy.spatial.distance.cosine. Сохраняем данные в массив в виде кортежа (номер предложения, расстояние)

In [7]:
proximity = []
for i in range(1, len(sentences)):
    cosine = distance.cosine(matrix[0], matrix[i])
    proximity.append((i, cosine))

Сортируем массив по возрастанию расстояния

In [8]:
proximity.sort(key=lambda touple:touple[1])

Выводим в файл номера двух самых близких по смыслу предложений к первому.

In [9]:
with open('SimilarityOfTexts_answer.txt', 'w') as file_obj:
    file_obj.write((str(proximity[0][0]) + ' ' + str(proximity[1][0])).strip('\n'))